## 完成对facebook数据的k近邻的机器学习，并通过网格搜索找到最佳的n_neighbors

In [7]:
# 导入必要的库
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime

# 读取Facebook位置数据集
print("读取Facebook位置数据集...")
fb_train = pd.read_csv('../data/FBlocation/train.csv')

# 将time列转换为年月日时分秒格式
print("将time列转换为年月日时分秒格式...")
# 假设time是Unix时间戳（以秒为单位）
fb_train['time_formatted'] = pd.to_datetime(fb_train['time'], unit='s')

# 从time_formatted提取星期几、小时和天作为新特征
print("从时间中提取星期几、小时和分钟作为新特征...")
fb_train['weekday'] = fb_train['time_formatted'].dt.dayofweek  # 星期几 (0-6, 0是星期一)
fb_train['hour'] = fb_train['time_formatted'].dt.hour  # 小时 (0-23)
fb_train['day'] = fb_train['time_formatted'].dt.day  # 天

# 显示数据集基本信息
print("数据集基本信息：")
print(f"样本数量: {fb_train.shape[0]}")
print(f"特征数量: {fb_train.shape[1]}")
print("\n数据集的前5行:")
print(fb_train.head(5))

# 检查缺失值
print("\n检查缺失值:")
print(fb_train.isnull().sum())

# 根据x和y特征列的数值筛选数据
print("\n根据x和y特征列筛选数据...")
# 筛选x和y都在特定范围内的数据
# 例如：选择x在0到3之间，y在0到3之间的数据点
# filtered_fb_train = fb_train[(fb_train['x'] >= 0) & (fb_train['x'] <= 3) & 
#                             (fb_train['y'] >= 0) & (fb_train['y'] <= 3)]
filtered_fb_train=fb_train.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")

print(f"筛选后的样本数量: {filtered_fb_train.shape[0]}")
print(f"筛选前的样本数量: {fb_train.shape[0]}")
print(f"筛选比例: {filtered_fb_train.shape[0]/fb_train.shape[0]:.2%}")


# 统计每个place_id的出现次数
print("\n统计每个place_id的出现次数...")
place_counts = filtered_fb_train['place_id'].value_counts()

# 找出出现次数大于3的place_id
valid_places = place_counts[place_counts > 3].index

# 只保留出现次数大于3的place_id对应的数据
print("去除place_id出现次数小于等于3的数据...")
filtered_fb_train = filtered_fb_train[filtered_fb_train['place_id'].isin(valid_places)]


# 提取特征和标签，加入新提取的时间特征
X = filtered_fb_train[['x', 'y', 'accuracy', 'weekday', 'hour', 'day']]  # 特征列
y = filtered_fb_train['place_id']  # 标签列
# 划分训练集和验证集
print("\n划分训练集和验证集...")
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.head(5))
# 特征标准化 - 使用训练集拟合标准化器，然后分别转换训练集和验证集
print("\n对特征进行标准化处理...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 对训练集进行fit和transform
X_val_scaled = scaler.transform(X_val)  # 对验证集只进行transform

# 打印数据集划分结果
print("Facebook位置数据集划分结果：")
print(f"训练集样本数量: {X_train.shape[0]}")
print(f"验证集样本数量: {X_val.shape[0]}")
print(f"特征数量: {X_train.shape[1]}")

# 统计标签数量
print("\n标签(place_id)的唯一值数量:", y.nunique())

print("\n数据预处理完成，可以进行KNN训练")

X_train_scaled[0:5]

读取Facebook位置数据集...
将time列转换为年月日时分秒格式...
从时间中提取星期几、小时和分钟作为新特征...
数据集基本信息：
样本数量: 29118021
特征数量: 10

数据集的前5行:
   row_id       x       y  accuracy    time    place_id      time_formatted  \
0       0  0.7941  9.0809        54  470702  8523065625 1970-01-06 10:45:02   
1       1  5.9567  4.7968        13  186555  1757726713 1970-01-03 03:49:15   
2       2  8.3078  7.0407        74  322648  1137537235 1970-01-04 17:37:28   
3       3  7.3665  2.5165        65  704587  6567393236 1970-01-09 03:43:07   
4       4  4.0961  1.1307        31  472130  7440663949 1970-01-06 11:08:50   

   weekday  hour  day  
0        1    10    6  
1        5     3    3  
2        6    17    4  
3        4     3    9  
4        1    11    6  

检查缺失值:
row_id            0
x                 0
y                 0
accuracy          0
time              0
place_id          0
time_formatted    0
weekday           0
hour              0
day               0
dtype: int64

根据x和y特征列筛选数据...
筛选后的样本数量: 17710
筛选前的样本数量: 29118021
筛

array([[-0.48293676,  0.90291955, -0.17915424,  1.13044561, -1.65117484,
        -0.76794871],
       [ 0.52467098,  0.46434201, -0.35200665, -1.25794776,  0.2182894 ,
         0.33718539],
       [-1.44515676, -1.31139703,  0.75788777,  1.13044561, -1.65117484,
        -0.76794871],
       [ 0.76717117, -1.75711752, -0.39749413, -0.06375108, -0.93215013,
         1.07394146],
       [ 1.0874789 , -1.74854597, -0.22464172, -0.06375108,  1.36872893,
         1.07394146]])

In [8]:
# 导入KNN分类器和评估指标
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

print("开始训练KNN模型...")

# 记录开始时间
start_time = time.time()

# 创建KNN分类器，使用默认K=5
knn = KNeighborsClassifier(n_neighbors=5)

# 训练模型
knn.fit(X_train_scaled, y_train)

# 记录结束时间
end_time = time.time()
print(f"KNN模型训练和评估总耗时: {end_time - start_time:.2f} 秒")


# 在验证集上进行预测
y_pred = knn.predict(X_val_scaled)

# 计算准确率
accuracy = accuracy_score(y_val, y_pred)
print(f"KNN模型准确率: {accuracy:.4f}")


print("\nKNN模型训练和评估完成！")


开始训练KNN模型...
KNN模型训练和评估总耗时: 0.01 秒
KNN模型准确率: 0.4805

KNN模型训练和评估完成！


In [9]:
# 导入GridSearchCV和交叉验证相关工具
from sklearn.model_selection import GridSearchCV

print("开始使用GridSearchCV进行KNN参数优化...")

# 设置开始时间
start_time = time.time()

# 定义参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance']
}

# 创建KNN分类器
knn = KNeighborsClassifier()

# 创建GridSearchCV对象，使用3折交叉验证
grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

# 在训练集上拟合网格搜索
grid_search.fit(X_train_scaled, y_train)

# 获取最佳参数和最佳得分
best_params = grid_search.best_params_
best_score = grid_search.best_score_


# 计算结束时间
end_time = time.time()
time_used = end_time - start_time

# 输出结果
print(f"\n网格搜索完成，耗时: {time_used:.2f} 秒")
print(f"最佳参数: {best_params}")
print(f"交叉验证最佳得分: {best_score:.4f}")


开始使用GridSearchCV进行KNN参数优化...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
/Library/Frameworks/Python.framework/Versions/3.1


网格搜索完成，耗时: 1.92 秒
最佳参数: {'n_neighbors': 11, 'weights': 'distance'}
交叉验证最佳得分: 0.4885


In [10]:
# 使用最佳参数构建模型
best_knn = KNeighborsClassifier(**best_params)
best_knn.fit(X_train_scaled, y_train)

# 在验证集上评估最佳模型
y_pred_best = best_knn.predict(X_val_scaled)
best_accuracy = accuracy_score(y_val, y_pred_best)
print(f"最佳模型在验证集上的准确率: {best_accuracy:.4f}")

最佳模型在验证集上的准确率: 0.5003
